In [22]:
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
from keras import optimizers
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout,LeakyReLU,Reshape
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras import utils as np_utils
from keras.layers import Input
from keras.models import Model
from keras.optimizers import adam
from keras.layers.normalization import BatchNormalization

train_path='C:/Users/jmlee/Desktop/Jimin/data_study/CNN_fruit/dataset/train_set'

class GAN():
    def __init__(self):
        self.img_rows = 200
        self.img_cols = 200
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

#         optimizer =keras.optimizers.Adam(0.0002, 0.5)

        # 식별자 생성 및 컴파일
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

        # 생성자 생성 및 컴파일
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer='adam')

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)  #Input 텐서 반환
        validity = model(img)  #타당성

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

#         # Load the dataset
#         (X_train, _), (_, _) = mnist.load_data()
#훈련데이터 불러와서 resize하고 저장

        X_train=[]
        for f in os.listdir(train_path):
            if f.split('.')[1] =='jpg':
                img = cv2.imread(train_path+'/'+f)
                img = cv2.resize(img,(200,200))
                img=  cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
                X_train.append(img)   #list형태
        X_train=np.array(X_train)

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
#         X_train = np.expand_dims(X_train,axis=3)  

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)  #전체이미지/2 크기의 랜덤 이미지인덱스 출력
            imgs = X_train[idx]  #랜덤한 이미지 

            noise = np.random.normal(0, 1, (half_batch, 100))  #노이즈

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)   #새로운 이미지 생성

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))  
            #train_on_batch(fit과 비슷)의 경우는 현재 전달받은 데이터를 모두 활용해서 gradient vector를 계산해서 업데이트함. 
            #return값이 score
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

                
    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gan/images/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_16 (Flatten)         (None, 120000)            0         
_________________________________________________________________
dense_81 (Dense)             (None, 512)               61440512  
_________________________________________________________________
leaky_re_lu_57 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_82 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_58 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 257       
Total params: 61,572,097
Trainable params: 61,572,097
Non-trainable params: 0
_________________________________________

C:\Users\jmlee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


InvalidArgumentError:  Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
	 [[node loss_26/sequential_26_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at C:\Users\jmlee\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_14113]

Function call stack:
keras_scratch_graph
